# DeepDream



## Importing libraries

In [6]:
from tensorflow.keras.applications import inception_v3
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tensorflow.keras.preprocessing import image

## Loading model

In [7]:
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, None, None, 3 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [ ]:
layer_contributions = {'mixed1' : 0.2, 'mixed2' : 2.0, 'mixed3' : 3.0, 'mixed6' : 0.2, 'mixed8' : 1.5, 'mixed10' : 2.1} # Tweak according to liking

## Defining loss

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.0)
for  layer_name in layer_contributions:
    coeff = layer_contributions(layer_name)
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

## Gradient Ascent

In [ ]:
dream = model.input()
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # Normalisation trick (Dividing by maximum MAE)
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('\t Loss value at ', i, ': ', loss_value)
        x += step * grad_values
    return x

## Running grad ascent over different octaves

In [ ]:
step = 0.01
num_octaves = 4
octave_scale = 1.4
iterations = 20
max_loss = 10.0
base_image_path = 'input_images/messi_2020.jpeg'
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shape[0])

for shape in successive_shapes:
    print('Processing image shape: ', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)

    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='messi_2020_final_dream.png')